In [2]:
role = sagemaker.get_execution_role()
session = sagemaker.Session()

## Setup Path 

In [3]:
# location for train.csv, val.csv and labels.csv
DATA_PATH = Path("../sm-data/")   

# Location for storing training_config.json
CONFIG_PATH = DATA_PATH/'config'
CONFIG_PATH.mkdir(exist_ok=True)

suffix = str(np.random.uniform())[4:9]

# S3 bucket name
bucket = 'toxic-pytorch-sagemaker-' + suffix

# Prefix for S3 bucket for input and output
prefix = 'toxic_comments/input'
prefix_output = 'toxic_comments/output'

In [4]:
!aws s3 mb s3://{bucket}

make_bucket: toxic-pytorch-sagemaker-74660


## Hyperparameters & Training Config

In [5]:
hyperparameters = {
    "epochs": 10,
    "lr": 8e-5,
    "max_seq_length": 512,
    "train_batch_size": 16,
    "lr_schedule": "warmup_cosine",
    "warmup_steps": 1000,
    "optimizer_type": "adamw"
}

In [6]:
training_config = {
    "run_text": "toxic comments",
    "finetuned_model": None,
    "do_lower_case": "True",
    "train_file": "train.csv",
    "val_file": "val.csv",
    "label_file": "labels.csv",
    "text_col": "comment_text",
    "label_col": '["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]',
    "multi_label": "True",
    "grad_accumulation_steps": "1",
    "fp16_opt_level": "O1",
    "fp16": "True",
    "model_type": "roberta",
    "model_name": "roberta-base",
    "logging_steps": "300"
}

with open(CONFIG_PATH/'training_config.json', 'w') as f:
    json.dump(training_config, f)

## Upload Data

In [7]:
# This is a helper feature to upload data
# from your local machine to S3 bucket.

s3_input = session.upload_data(DATA_PATH, bucket=bucket , key_prefix=prefix)

session.upload_data(str(DATA_PATH/'val.csv'), bucket=bucket , key_prefix=prefix)

's3://toxic-pytorch-sagemaker-74660/toxic_comments/input/val.csv'

In [8]:
session.upload_data(str(DATA_PATH/'labels.csv'), bucket=bucket , key_prefix=prefix)

's3://toxic-pytorch-sagemaker-74660/toxic_comments/input/labels.csv'

In [9]:
session.upload_data(str(DATA_PATH/'train.csv'), bucket=bucket , key_prefix=prefix)

's3://toxic-pytorch-sagemaker-74660/toxic_comments/input/train.csv'

## Create an Estimator and start training

In [10]:
#account = session.boto_session.client('sts').get_caller_identity()['Account']
#region = session.boto_session.region_name

#image = "{}.dkr.ecr.{}.amazonaws.com/sagemaker-bert:1.0-gpu-py36".format(account, region)
image = "111652037296.dkr.ecr.us-west-2.amazonaws.com/chazarey-sagemaker-fast-bert:1.0-gpu-py36"
#TODO Convert this to using SM Pytorch 

In [11]:
output_path = "s3://{}/{}".format(bucket, prefix_output)

In [12]:
estimator = sagemaker.estimator.Estimator(image, 
                                          role,
                                          train_instance_count=1, 
                                          train_instance_type='ml.p3.8xlarge', 
                                          output_path=output_path, 
                                          base_job_name='toxic-comments',
                                          hyperparameters=hyperparameters,
                                          sagemaker_session=session
                                         )

In [13]:
estimator.fit(s3_input)

2019-11-18 05:23:42 Starting - Starting the training job...
2019-11-18 05:23:49 Starting - Launching requested ML instances......
2019-11-18 05:24:52 Starting - Preparing the instances for training......
2019-11-18 05:26:07 Downloading - Downloading input data
2019-11-18 05:26:07 Training - Downloading the training image..............Starting the training.
/opt/ml/input/data/training/config/training_config.json
{'run_text': 'toxic comments', 'finetuned_model': None, 'do_lower_case': 'True', 'train_file': 'train.csv', 'val_file': 'val.csv', 'label_file': 'labels.csv', 'text_col': 'comment_text', 'label_col': '["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]', 'multi_label': 'True', 'grad_accumulation_steps': '1', 'fp16_opt_level': 'O1', 'fp16': 'True', 'model_type': 'roberta', 'model_name': 'roberta-base', 'logging_steps': '300'}
{'train_batch_size': '16', 'warmup_steps': '1000', 'lr': '8e-05', 'max_seq_length': '512', 'optimizer_type': 'adamw', 'lr_schedule': '

## Deploy the model to hosting service

In [14]:
predictor = estimator.deploy(1, 
                             'ml.m5.large', 
                             endpoint_name='bert-toxic-comments', 
                             serializer=json_serializer)

--------------------------------------------------------------------------------------------------------------!

In [15]:
### Invoke the Endpoint
client = boto3.client('sagemaker-runtime')

sample_payload='{"text": "this is really really good thanks for recommending!!"}'

response = client.invoke_endpoint(
    EndpointName='bert-toxic-comments',
    Body=sample_payload,
    ContentType='application/json'
)
print('Our result for this payload is: {}'.format(response['Body'].read().decode('ascii')))

Our result for this payload is: [["toxic", 9.29754605749622e-05], ["insult", 4.071386865689419e-05], ["obscene", 3.189296694472432e-05], ["severe_toxic", 2.644667256390676e-05], ["identity_hate", 1.8441571228322573e-05], ["threat", 1.307918591919588e-05]]
